In [49]:
import numpy as np

### How the edge features of a graph are aggregated along with the node features in GNN

* Let's say we have a graph G(V,E), where V represents the nodes and E represents the edges of the graph. 

 $$ \text{node features} = 
\begin{bmatrix} 
\mathbf{x_1} & ... & \mathbf{x_n}\\
\end{bmatrix}
$$

* If the size of the node features vector is $d^n$, the node features array as shape of $(N,d^n)$ with N is the number of nodes in the graph G
$$\text{edge features} = 
\begin{bmatrix} 
\mathbf{x_{e_{1,1}}} & ... & \mathbf{x_{e_{1,n}}}\\
\vdots & \ddots & \vdots \\
\mathbf{x_{e_{n,1}}} & ... & \mathbf{x_{e_{n,n}}}\\
\end{bmatrix}
$$

* If the size of the edge features vector is $d^e$, the node features array as shape of $(N,N,d^e)$

$$\text{combined features} = \text{edge features} + \text{node features} = 
\begin{bmatrix} 
\mathbf{x_{e_{1,1}}} + \mathbf{x_1} & ... & \mathbf{x_{e_{1,n}}} + \mathbf{x_n}\\
\vdots & \ddots & \vdots \\
\mathbf{x_{e_{n,1}}} + \mathbf{x_1} & ... & \mathbf{x_{e_{n,n}}} + \mathbf{x_n}\\
\end{bmatrix}
$$

* Then the size of the combined  features array has shape of $(N,N,d^e + d^n)$. Note for this we need to have the shape of $d^n$ = $d^e$
$$
\text{masked features} = A \circ \text{combined features} = 
\begin{bmatrix} 
a_{1,1}(\mathbf{x_{e_{1,1}}} + \mathbf{x_1}) & ... & a_{1,n}(\mathbf{x_{e_{1,n}}} + \mathbf{x_n})\\
\vdots & \ddots & \vdots \\
a_{n,1}(\mathbf{x_{e_{n,1}}} + \mathbf{x_1}) & ... & a_{n,n}(\mathbf{x_{e_{n,n}}} + \mathbf{x_n})\\
\end{bmatrix}
$$

* To select the edge features present in the graph we need to mask the above array with Adjacency matrix $A$


$$
\begin{bmatrix} 
a_{1,1}(\mathbf{x_{e_{1,1}}} + \mathbf{x_1}) + ...+ a_{1,n}(\mathbf{x_{e_{1,n}}} + \mathbf{x_n})\\
\vdots \\
a_{n,1}(\mathbf{x_{e_{n,1}}} + \mathbf{x_1}) + ... + a_{n,n}(\mathbf{x_{e_{n,n}}} + \mathbf{x_n})\\
\end{bmatrix}
$$

* Finally we now reduce this matrix by summing along the rows to perform the aggregation (of the messages at current node) operation 


In [50]:
N = 3 # number of nodes 
d = 2 # the dimention of node (edge) features 

# the node feature array: shape(A) = N,d
A = np.random.randint(0,10,(N,d))
#A = np.expand_dims(A,axis=1)
A


array([[0, 6],
       [1, 4],
       [7, 5]])

In [51]:
# the edge faeture array: shape(B) = N,N,d
B = np.random.randint(0,10,(N,N,d))
B

array([[[3, 2],
        [9, 4],
        [1, 3]],

       [[8, 9],
        [2, 6],
        [1, 4]],

       [[3, 7],
        [0, 3],
        [9, 0]]])

In [52]:
A + B

array([[[ 3,  8],
        [10,  8],
        [ 8,  8]],

       [[ 8, 15],
        [ 3, 10],
        [ 8,  9]],

       [[ 3, 13],
        [ 1,  7],
        [16,  5]]])

In [53]:
# adjacancy matrix: shape(N,N)
Ad =  np.random.randint(0,2,(N,N))
Ad

array([[1, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [54]:
# lets rehsape Adjacancy matrix to have shape of: (N,N,1)
M = np.expand_dims(Ad,axis=-1)
M

array([[[1],
        [0],
        [1]],

       [[0],
        [0],
        [1]],

       [[0],
        [0],
        [1]]])

In [55]:
# lets add Node features and Edge features together 
# A: (N,d), B: (N,N,d) , with broadcasting A: (1,N,d), B: (N,N,d) 
# so A + B : (N,N,d)
C = A + B 
C

array([[[ 3,  8],
        [10,  8],
        [ 8,  8]],

       [[ 8, 15],
        [ 3, 10],
        [ 8,  9]],

       [[ 3, 13],
        [ 1,  7],
        [16,  5]]])

In [56]:
# lets use the mask to select the (node + edge feature) only in the graph connected nodes (i.e. edges)

MC = M * C
MC

array([[[ 3,  8],
        [ 0,  0],
        [ 8,  8]],

       [[ 0,  0],
        [ 0,  0],
        [ 8,  9]],

       [[ 0,  0],
        [ 0,  0],
        [16,  5]]])

In [57]:
# lets aggrigate node and edge feature 
# F: shape (N,d)
F = np.sum(MC, axis=1)
F

array([[11, 16],
       [ 8,  9],
       [16,  5]])

Broadcasting examples 

In [58]:
u = np.random.randint(0,10,(3,2))
v = np.random.randint(0,10,(3,3,2))

In [59]:
u


array([[3, 4],
       [2, 9],
       [7, 8]])

In [60]:
v

array([[[3, 6],
        [4, 1],
        [9, 2]],

       [[4, 7],
        [4, 0],
        [7, 3]],

       [[4, 9],
        [7, 3],
        [5, 1]]])

In [61]:
# here u will sum along 0 th axis (along rows)
u + v

array([[[ 6, 10],
        [ 6, 10],
        [16, 10]],

       [[ 7, 11],
        [ 6,  9],
        [14, 11]],

       [[ 7, 13],
        [ 9, 12],
        [12,  9]]])

In [62]:
# uu shape : (n,1,d)
uu = np.expand_dims(u,axis=1)
uu

array([[[3, 4]],

       [[2, 9]],

       [[7, 8]]])

In [64]:
v

array([[[3, 6],
        [4, 1],
        [9, 2]],

       [[4, 7],
        [4, 0],
        [7, 3]],

       [[4, 9],
        [7, 3],
        [5, 1]]])

In [63]:
# now the summision is along 1st axis (along columns)
uu + v

array([[[ 6, 10],
        [ 7,  5],
        [12,  6]],

       [[ 6, 16],
        [ 6,  9],
        [ 9, 12]],

       [[11, 17],
        [14, 11],
        [12,  9]]])